In [94]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os
import csv
from sklearn.preprocessing import RobustScaler

#3.1.1 Logistic Regression
#http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#h = .02  # step size in the mesh
def get_data(name = ''):
    df = ""
    path = os.getcwd()
    direct='data'
    file=os.path.join(path,direct, name)
    #opencsv=""
    try:
        opencsv = open(os.path.join(path,direct,name), 'r')
        print(opencsv)#find the race-result-horse file
    except:
        while opencsv != name:  # if the file cant be found if there is an error
            print("Could not open ", "file")
            opencsv = input("\nPlease try to open file again: ")
    else:
        with open(os.path.join(path,direct,name)) as f:
            feature_names = []#f.readline()
            #print(feature_names)
            reader = csv.reader(f)
            for row in reader:
                feature_names = row 
                #print(feature_names)
                break
            df =  pd.read_csv(open(os.path.join(path,direct,name)), header=None, names = feature_names) #, na_values = ['WV-A' , 'WV'])
            df.drop(0, inplace = True)
    return df

def process(df = pd.DataFrame({}), a=[]):
    for i in range(5864):
        if a[i] == 1:
            i=i+1
            print("Hi my frd")
            df.loc[i, 'HorseWin'] = 1
            df.loc[i, 'HorseRankTop3'] = 1
            df.loc[i, 'HorseRankTop50Percent'] = 1
        elif a[i] == 2 or a[i] == 3:
            i=i+1
            df.loc[i, 'HorseWin'] = 0
            df.loc[i, 'HorseRankTop3'] = 1
            df.loc[i, 'HorseRankTop50Percent'] = 1
        elif a[i] == 4 or a[i] == 5 or a[i] == 6:
            i=i+1
            df.loc[i, 'HorseWin'] = 0
            df.loc[i, 'HorseRankTop3'] = 0
            df.loc[i, 'HorseRankTop50Percent'] = 1
        else:
            i=i+1
            df.loc[i, 'HorseWin'] = 0
            df.loc[i, 'HorseRankTop3'] = 0
            df.loc[i, 'HorseRankTop50Percent'] = 0
    return df

df1 = get_data('training.csv')
df2 = get_data('testing.csv')
df1.recent_ave_rank = df1.recent_ave_rank.astype(int)
df1["HorseWin"] = ""
df1["HorseRankTop3"] = ""
df1["HorseRankTop50Percent"] = ""
df2["HorseWin"] = ""
df2["HorseRankTop3"] = ""
df2["HorseRankTop50Percent"] = ""
#print (df1)
#print (df2)
#df1 = process(df1)
#print (df1)

#19: RaceID; 4: HorseID; 21: RecentAvg rank
#24: JockeyAbg rank; 26: TrainerAvg rank
X_train = df1.loc[:,['jockey_ave_rank','trainer_ave_rank','actual_weight','declared_horse_weight']] #24,26
Y_train = df1.loc[:,['recent_ave_rank']]
#print(np.ravel(Y_train)) #no problem: not only have "7"

X_test = df2.loc[:,['jockey_ave_rank','trainer_ave_rank','actual_weight','declared_horse_weight']] #no problem
#print(X_train)

print("Logistic----------------------------------------------------------------------------------")
lr_model = linear_model.LogisticRegression(C=1e5)
lr_model.fit(X_train, np.ravel(Y_train)) 
lr_pre = lr_model.predict(X_test)
print(lr_pre) #problem here: prediction result have "7" only
#print(len(lr_pre))
df2 = process(df2,lr_pre)

#print (df2)
lr_result = df2.iloc[:,[19,4,28,29,30]]
#print(lr_result)

print("Gaussian------------------------------------------------------------------------------------")
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(X_train, np.ravel(Y_train))
GaussianNB(priors=None)
nb_pre = nb_model.predict(X_test)
print(nb_pre)
#for i in nb_pre:
#    print(i)
df2 = process(df2,nb_pre)
nb_result = df2.iloc[:,[19,4,28,29,30]]
#print(nb_result)

"""from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train, Y_train)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
nb_pre = nb_model.predict(X_test)

from sklearn.naive_bayes import BernoulliNB
nb_model = BernoulliNB()
nb_model.fit(X_train, Y_train)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
nb_pre = nb_model.predict(X_test)"""

print("SVM------------------------------------------------------------------------------------")
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train, np.ravel(Y_train))
#Example:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
svm_pre = svm_model.predict(X_test)
print(svm_pre)
df2 = process(df2,svm_pre)
svm_result = df2.iloc[:,[19,4,28,29,30]]
#print(svm_result)

print("Random Forest-----------------------------------------------------------------------------")
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
rf_model = RandomForestClassifier(max_depth=2, random_state=0)
rf_model.fit(X_train, np.ravel(Y_train))
#Example:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
#rf_model.feature_importances_
rf_pre = rf_model.predict(X_test)
print(rf_pre)
df2 = process(df2,rf_pre)
rf_result = df2.iloc[:,[19,4,28,29,30]]
#print(rf_result)




<_io.TextIOWrapper name='C:\\Users\\Kelvin-PC\\OneDrive - The Chinese University of Hong Kong\\Documents\\Github\\CSCI3320_PROJECT\\csci3320_prj\\data\\training.csv' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='C:\\Users\\Kelvin-PC\\OneDrive - The Chinese University of Hong Kong\\Documents\\Github\\CSCI3320_PROJECT\\csci3320_prj\\data\\testing.csv' mode='r' encoding='cp1252'>
Logistic----------------------------------------------------------------------------------
[7 7 7 ..., 7 7 7]
Gaussian------------------------------------------------------------------------------------
[7 7 7 ..., 7 7 7]
SVM------------------------------------------------------------------------------------
[7 5 7 ..., 7 7 7]
Random Forest-----------------------------------------------------------------------------
[7 7 7 ..., 7 7 7]
